In [1]:
# 라이브러리 호출
import pandas as pd

#### 본 사용자 매뉴얼에서는 VAR 모형과 추정에 대한 기본적인 내용을 다룬다.
- 최적시차 설정
- 축약형 VAR 모형 추정

## 1. 데이터 불러오기
- 작성자: 고려대학교 경제학과 강규호 교수, 데이터사이언스팀 이창훈 과장

데이터는 한국의 전기대비 실질 GDP 성장률, CD91일 금리, 인플레이션율 소비자 가격 지수 (Consumer Price Index, CPI), CD91일 금리로 1999년 4분기부터 2023년 4분끼지의 분기별 자료이다. 해당 데이터는 ECOS (https://ecos.bok.or.kr/#/)에서 다운로드 받았으며 컴퓨터에 "python_data.xlsx"라는 이름의 엑셀파일의 "KORmacro"라는 시트에 저장되어 있다. 이때, 인플레이션율의 경우 소비자 가격 지수 (Consumer Price Index, CPI)의 전년동기대비 차분 (YoY)을 통해 미리 변환한 것이다.

`data` 폴더에서 `python_data.xlsx`를 불러온 후 파이썬 표준 자료형인 `pandas.DataFrame`으로 전처리를 수행한다. `DataFrame`은 테이블 형태의 데이터의 행(날짜)과 열(변수명) 정보를 가지고 있어, 행렬이나 벡터형태로 분석하는 것보다 유용하다. `df`라는 이름으로 할당하는 것이 일반적이다. 데이터프레임에 익숙하지 않은 경우 아래 셀의 코드를 줄별로 여러 셀로 나누어 실행해보는 것을 추천한다. 원천기관에서 raw 데이터를 불러오고 전처리하는 과정이 복잡해 보일 수 있지만, 한번 해두면 크게 코드를 수정할 필요없이 새로운 데이터에 대해서도 분석 가능한 형태로 전처리 할 수 있기 때문에 장기적으로 효율적인 방법이다.

In [3]:
df = pd.read_excel('../data/python_data.xlsx', sheet_name='KORmacro') # 데이터 엑셀파일 불러오기
df = df.set_index('date') # date 명의 열을 인덱스로 지정한다.
df.index.name = None # 인덱스 이름 date를 제거한다.
df.index = df.index.str.replace('/', '-') # 2000/03 형식을 pandas가 날짜로 인식하도록 2000-3형식으로 변환한다.
df.index = pd.to_datetime(df.index) + pd.offsets.QuarterEnd(0) # 인덱스(행)를 날짜로 인식하도록 변환한다.
df.index.freq = pd.offsets.QuarterEnd() # 인덱스의 frequency를 분기말(2000-03-31)로 설정한다.
column_order = ['inf', 'rgdp', 'cd91'] # df의 열 순서를 리스트로 미리 할당
df = df[column_order] # df의 열 순서를 column_order에 맞게 변경
df

C:\Users\BOK\AppData\Local\Temp\ipykernel_118520\1441099652.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index) + pd.offsets.QuarterEnd(0) # 인덱스(행)를 날짜로 인식하도록 변환한다.


,inf,rgdp,cd91
2000-12-31,2.508257,-0.3,6.97
2001-03-31,3.627876,1.2,5.95
2001-06-30,4.919778,1.3,5.78
2001-09-30,4.158456,1.4,4.97
2001-12-31,3.244327,1.8,4.58
...,...,...,...
2022-12-31,5.083054,-0.3,3.91
2023-03-31,4.496285,0.3,3.64
2023-06-30,3.212650,0.6,3.63
2023-09-30,3.079117,0.6,3.74


## 2. VAR 모형의 최적 시차 결정

`bok_da` 라이브러리 시계열분석 패키지(`ts`)의 VAR모형 서브패키지(`var`)에 있는 `order_var`함수를 사용하면 VAR 모형의 최적 시차를 추정 할 수 있다.

#### (예시)
위에서 불러온 df는 분기별 데이터로 구성되어 있기 때문에 VAR 모형의 예상 최대 시차를 lag_max=4이라고 가정한다. 참고로 함수 인자에 verbose=Fasle로 설정하는 경우, 분석 결과를 출력하지 않는다.

함수를 사용할 때, `bd.ts.var.order_var`과 같이 사용하는 방법외에 `var` 모듈에서 `order_var` 함수를 미리 불러온 뒤에 사용할 수도 있다. 같은 함수를 여러번 사용하는 경우 이렇게 사용하는 방식이 편하다. 

In [4]:
from bok_da.ts.var import order_var
lag = order_var(df, lag_max=4, verbose=True)

   
 VAR Order Selected by Sequential testing =  2
 VAR Order Selected by AIC =                 2
 VAR Order Selected by BIC =                 2
 VAR Order Selected by HQ =                  2


아래 코드와 같이 lag 인스턴스의 description 프로퍼티를 이용해 결과물에 대한 설명을 볼 수있다.

In [5]:
print(lag.description)

seq = 순차적 검정으로 추정된 VAR(p) 모형의 최적 시차
aic = Akaike 정보기준 (AIC)로 추정된 VAR(p) 모형의 최적 시차
bic = Bayesian 정보기준 (BIC)로 추정된 VAR(p) 모형의 최적 시차
hq = Hannan-Quinn 정보기준 HQIC)로 추정된 VAR(p) 모형의 최적 시차
aic_value = AIC 값
bic_value = AIC 값
hq_value = AIC 값



lag 인스턴스를 이용해 추정결과를 보는 방법은 lag.bic, lag.bic_value를 실행하면된다

In [6]:
print(lag.bic)

2


In [7]:
print(lag.bic_value)

[[-3.29722326]]


추정 결과, 모든 검정 결과가 VAR 모형의 최적시차를 $\hat{p}=2$로 추정하고 있다는 것을 확인할 수 있다. 이런 경우는 특수한 경우로 보통 네 가지 결과는 각기 다른 값을 추정하는 것이 일반적이다. 예를 들어, lag_max=8의 경우 다음과 같은 추정 결과를 얻을 수 있다.

In [8]:
lag = order_var(df, lag_max=8, verbose=True)

   
 VAR Order Selected by Sequential testing =  8
 VAR Order Selected by AIC =                 8
 VAR Order Selected by BIC =                 1
 VAR Order Selected by HQ =                  2


위와 같이 각 검정이 다른 결과를 가르킬 경우 각 방법의 장단점을 연구자가 고려하여 택일할 필요가 있다. 우선, 순차적 검정은 실제로 참인 귀무가설을 잘못 기각하는 과대 식별 (over-specification)의 확률이 존재한다. 즉, 귀무가설을 기각하지 못하고 시차를 낮추어 다시 검정해야하는 경우임에도 귀무가설을 잘못하여 기각하고 순차적 검정을 멈추는 문제가 발생할 수 있다.

다음으로, 정보 기준 관련해서 일치성 (consistency)이 성립하는 BIC나 HQ와 달리 AIC는 inconsistent하여 VAR 모형의 시차를 과대 식별할 수 있다는 한계가 존재한다. 어떤 시차를 사용하든 유사한 결과가 도출되는 경우가 가장 이상적이지만, 만약 네 가지 결과값이 다르다면 BIC 또는 HQ에서 추정된 최적 시차를 사용하는 할 것을 권한다.

## 3. 축약형 VAR 모형 추정

#### (참고) 축약형 VAR 모형
VAR 모형은 아래와 같이 내생변수들 사이의 관계를 나타내는 구조 VAR 모형 (structural-form VAR model)
$\begin{align}
    & B_{0}y_{t} = B_{1}y_{t-1} + \cdots + B_{p}y_{t-p}+e_{t} \notag \\
    & \iff B(L)y_{t} = e_{t}\text{ where }e_{t}\sim WN(0,\Omega) \notag
\end{align}$
과 내생변수를 외생변수들로 나타내는 축약형 VAR 모형 (reduced-form VAR model)
$\begin{align}
    & y_{t} = A_{1}y_{t-1} + \cdots + A_{p}y_{t-p} + u_{t} \notag \\
    & \iff A(L)y_{t} = u_{t}\text{ where }u_{t}\sim WN(0,\Sigma) \notag
\end{align}$
로 나타낼 수 있다. 이때, 구조 충격 $e_{t}$의 분산 $\Omega$는 대각행렬이며, 축약형 VAR 모형은 구조형 VAR 모형 전체에 $B_{0}^{-1}$을 곱한 형태이기 때문에 축약형 충격 $u_{t}$의 분산인 $\Sigma$는 반드시 대각행렬이 되지 않는다.

두 형태 중에서 VAR 모형을 사용한 시계열 분석의 목적이 되는 형태는 구조형 VAR 모형이다. 하지만 구조형 VAR 모형은 시스템을 구성하는 각 식 별로 특정 내생변수를 구성하는 다른 내생변수들이 오차항과 연관된 내생성 문제가 존재한다는 한계가 있다. 따라서 일반적으로 VAR 모형을 사용하는 연구자는 축약형 VAR 모형을 추정하고, 경제학적인 이론에 기반하여 구조형 모형을 식별하는 방법을 선택한다. 이때, 축약형 VAR 모형의 각 방정식은 내생변수가 없고 모두 동일한 설명변수(각 시점의 자기회귀항)를 사용하기 때문에, Seemingly Unrelated Regressions-GLS로 효율을 높이려 해도 OLS와 결과가 같다(Hamilton (1994)). 따라서 GLS 방법이 아닌 OLS 방법을 사용하여 추정하여도 충분하다.

축약형 VAR 모형의 내생변수 $y_{t}$가 $k$개의 변수로 구성된 다음과 같은 벡터라고 하자.
$\begin{equation}
    y_{t} = \begin{pmatrix}
    y_{1t} \\
    \vdots \\
    y_{kt}
    \end{pmatrix} \notag
\end{equation}$
이 경우 축약형 VAR 모형은 다음과 같은 선형회귀 모형으로 표현할 수 있다.
$\begin{align}
    & \underbrace{y_{t}}_{k\times 1} = \underbrace{v}_{k\times 1} + \underbrace{A_{1}}_{k\times k}\underbrace{y_{t-1}}_{k\times 1} + \cdots + \underbrace{A_{p}}_{k\times k}\underbrace{y_{t-p}}_{k\times 1} + \underbrace{u_{t}}_{k\times 1} \notag \\
    & \Rightarrow y_{t}^{\prime} = v^{\prime} + y_{t-1}^{\prime}A_{1}^{\prime} + \cdots + y_{t-p}^{\prime}A_{p}^{\prime} + u_{t}^{\prime} \notag \\
    & \Rightarrow y_{t}^{\prime} = (1, y_{t-1}^{\prime},\cdots,y_{t-p}^{\prime})\begin{pmatrix}
        v^{\prime} \\
        A_{1}^{\prime} \\
        \vdots \\
        A_{p}^{\prime}
    \end{pmatrix} + u_{t}^{\prime} \notag \\
    & \Rightarrow y_{t}^{\prime} = Z_{t}^{\prime}B + u_{t}^{\prime} \notag
\end{align}$
위 식을 각 $t=p+1,\cdots,T$까지의 시기에 대해 쌓아올린다면 다음과 같이 표현할 수 있다.
$\begin{align}
    & \Rightarrow \begin{pmatrix}
        y_{p+1}^{\prime} \\
        \vdots \\
        y_{T}^{\prime}
    \end{pmatrix} = \begin{pmatrix}
        1 & y_{p}^{\prime} & \cdots & y_{1}^{\prime} \\
        \vdots & \vdots & \ddots & \vdots \\
        1 & y_{T-1}^{\prime} & \cdots & y_{T-p}^{\prime}
    \end{pmatrix}B  + \begin{pmatrix}
        u_{p+1}^{\prime} \\
        \vdots \\
        u_{T}^{\prime}
    \end{pmatrix} \notag \\
    & \Rightarrow Y = Z\cdot B + U\text{ where }Z = \begin{pmatrix}
        Z_{p+1}^{\prime} \\
        \vdots \\
        Z_{T}^{\prime}
    \end{pmatrix} \notag
\end{align}$
따라서, OLS 추정량인 $\hat{B}$는 다음과 같이 각 식에 대한 OLS 추정량을 모아놓은 벡터가 된다.
$\begin{align}
    \hat{B} & = (Z^{\prime}Z)^{-1}Z^{\prime}Y \notag \\
    & = [(Z_{1}^{\prime}Z_{1})^{-1}Z_{1}^{\prime}Y_{1},\cdots,(Z_{k}^{\prime}Z_{k})^{-1}Z_{k}^{\prime}Y_{k}] \notag \\
    & = [\hat{\beta}_{1},\cdots,\hat{\beta}_{k}] \notag
\end{align}$
나아가 선형회귀 식의 잔차는
$\begin{equation}
    \hat{U} = Y - Z\cdot\hat{B} \notag
\end{equation}$
으로 도출할 수 있기 때문에, 분산-공분산 행렬에 대한 추정량은 다음과 같다.
$\begin{equation}
    \hat{\Sigma} = \frac{1}{T-p}\hat{U}^{\prime}\hat{U} = \frac{1}{T-p}\sum_{t=p+1}^{T}{\hat{u}_{t}\hat{u}_{t}^{\prime}} \notag
\end{equation}$

#### (예시)

`bok_da` 라이브러리 시계열분석 패키지(`ts`)의 VAR모형 서브패키지(`var`)에 있는 `var`함수를 사용하면 축양형 VAR 모형을 추정 할 수 있다.

 `var` 함수 코드에는 VAR(p) 모형의 절편항을 고려하지 않기 위해 입력되는 `y` 변수에 대해 자동으로 demeaning 과정이 들어가 있다. 따라서, 예측 시 demeaning 과정에서 제거된 mean을 예측값에 더해주도록 코드가 작성되어 있다.

또한, VAR 모형의 시차를 입력하지 않을 경우 이전에 설명한 `order_var` 함수에서 표본크기에 기반한 `pmax`를 사용하여 최적 시차를 추정하여 사용한다. 구체적으로 Ng and Perron (2001)에서 제안하는 rule of thumb인 다음과 같은 수식에 따라 계산되도록 자동화하였다.
$\begin{equation}
    lag\_max = round\Bigg(12\cdot(\frac{T}{100})^{\frac{1}{4}}\Bigg) \notag
\end{equation}$

이때, `lag_max`를 입력하여 계산된 다양한 최적시차 추정량 중에서 일치추정량인 BIC를 default로 사용한다. 만약 자동화된 절차에 따라 추정된 BIC가 아닌 다른 최적 시차(lag)를 사용하고 싶다면, (1) 연구자가 임의로 결정한 최적시차를 lag이라는 입력값으로 넣거나, (2) 위 3절에서 설명한 것과 같이 연구자가 임의로 결정한 최대 시차를 `order_var`에 입력하여 추정된 네 가지 최적 시차 추정량 중에서 하나를 lag이라는 입력값으로 넣으면 된다.

In [9]:
from bok_da.ts.var import var

res = var(df) # lag 인자를 입력하지 않는 경우 자동으로 시차 설정

추정 결과 res 인스턴스는 다음의 결과를 포함한다.
</br>
    `phi_hat`: 축약형 VAR(p) 모형의 계수 행렬 추정량
    </br>
    `omega_hat`: 축약형 VAR(p) 모형의 분산-공분산 행렬 추정량
    </br>
    `f_mat`: 축약형 VAR(p) 모형의 동반행렬 (Companion Matrix) 형태의 계수 행렬 추정량
    </br>
    `u_hat`: 축약형 VAR(p) 모형의 잔차
    </br>
    `y0`: 추정에 사용된 반응변수 행렬
    </br>
    `y_lag`: 추정에 사용된 설명변수 행렬
    </br>
    `y_pred`: 예측치
    </br>
    `lag`: 모형에 사용되 시차
    

In [10]:
res.phi_hat

array([[ 0.87590527, -0.14849765,  0.09139085],
       [ 0.06716272,  0.20447624,  0.21538759],
       [ 0.03837978,  0.11803926,  0.86995709]])

In [11]:
res.omega_hat

array([[0.347869  , 0.07532279, 0.04602065],
       [0.07532279, 0.80138552, 0.02408009],
       [0.04602065, 0.02408009, 0.10386358]])

다음과 같이 lag=2로 시차를 직접 입력할 수도 있다.

In [12]:
res = var(df, lag=2)

In [13]:
res.phi_hat

array([[ 1.08270465, -0.56234252,  0.05586755],
       [ 0.02825777,  0.30198823,  0.19871594],
       [ 0.05155508, -0.37952045,  1.12559623],
       [-0.21208464,  0.53987643,  0.02006422],
       [ 0.13964086,  0.04582516,  0.00519975],
       [-0.04770968,  0.40919605, -0.2407622 ]])

In [14]:
res.omega_hat

array([[0.31014499, 0.13152801, 0.0431908 ],
       [0.13152801, 0.70564407, 0.04113348],
       [0.0431908 , 0.04113348, 0.09788613]])

참고로 함수를 사용하는 방법 외에 VectorAutoRegression 클래스를 이용해서 추정할 수도 있다.

`bok_da` 라이브러리 시계열분석 패키지(`ts`)의 VAR모형 서브패키지(`var`)에서 `VectorAutoRegression` 클래스를 `VAR`로 불러온다.

In [15]:
from bok_da.ts.var import VectorAutoRegression as VAR

In [16]:
model = VAR(lag=2)
res = model.fit(df)

다음과 같이 함수를 사용하는 방법과 동일한 결과를 나타낸다.

In [17]:
for i in range(res.lag):
    print(pd.DataFrame(res.phi_hat[i]).T)

          0         1         2
0  1.082705 -0.562343  0.055868
          0         1         2
0  0.028258  0.301988  0.198716


함수를 통해 도출된 "phi_hat"이라는 변수는 위에서 설명된 $\hat{B}$를 의미한다. 이때, 절편항이 제외된 축약형 VAR 모형에서 $\hat{B}$는 아래와 같다.
$\begin{equation}
    \hat{B} = \begin{pmatrix}
        \hat{A}_{1}^{\prime} \\
        \vdots \\
        \hat{A}_{p}^{\prime}
    \end{pmatrix} \notag
\end{equation}$
따라서 축약형 VAR 모형에서 $A_{1}$부터 $A_{p}$까지의 계수 행렬을 보기 위해서는 "phi_hat"을 전치행렬 (transpose)로 바꿔주면 된다.